# SpinToolkit

Demo code for Spin wave theory for complex spin orders I: linear spin wave, generalized linear spin wave, multi-magnon continuum and finite-temperature effects_

Authors: Lei Xu, Xiaojian Shi, Yangjie Jiao, and [Zhentao Wang](https://orcid.org/0000-0001-7442-2933)

In this demo, we compute the finite-$T$ multi-magnon excitation of the spin-$1/2$ XXZ model on triangular lattice

## Prerequisites

import the modules and set the number of threads for parallelling

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from joblib import Parallel, delayed
import SpinToolkit_py as sptk

In [ ]:
# check the system information and choose the number of threads accordingly
sptk.print_system_info()

n_jobs = 20 # set manually
# n_jobs = n_jobs   # use the maximum number of threads available

## Define lattice and hamiltonian

- triangular lattice with a 3-sublattice supercell compatible with 120$^\circ$ order

- XXZ model

- use "dipole" mode

In [ ]:
latt = sptk.lattice(filename = "triangular_K.toml")
Nsites = latt.Nsites()

In [ ]:
J = 1.0  # nearest-neighbor exchange
Δ = 0.9  # exchange anisotropy
h = 0.3  # Zeeman field

hamiltonian = sptk.model_spin(S = 0.5, mode = "dipole", lattice = latt)
print()

for site_i in range(Nsites):
        coor_i, sub_i = latt.site2coor(site = site_i)
        coor0_i, Mi   = latt.coor2supercell0(coor = coor_i)
        xi = coor_i[0]
        yi = coor_i[1]

        # 1st neighbor terms
        coor_j      = [xi + 1, yi]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi + 1, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        # Zeeman field
        hamiltonian.add_zeeman(h = sptk.Vec3(0.0, 0.0, h), site = site_i)

hamiltonian.simplify()
print()
hamiltonian.build_mc_list()
print()

## Perform minimization (minimize from several independent initial random guesses, then pick the lowest-energy solution)

In [ ]:
print("Optimizing ground state", end = "... ", flush = True)
start        = time.perf_counter()
s_min, f_min = hamiltonian.optimize_spins_dipole(total_seeds = 20, seed0 = 5) # more seeds -> more likely to hit global minimum
end          = time.perf_counter()
print(f"{end - start}s")
print()

print(f"f_min (global): {f_min}")
for i in range(Nsites):
    print(f"site-{i}: {s_min[i]}")

## Initialize the LSW calculation and prepare the rotation matrices $\vec{s_{i}} = R_{i} {\hat z}$

In [ ]:
R = hamiltonian.init_LSW(s = s_min)
for i in range(Nsites):
    print(f"site-{i}:")
    print(R[i])
    print()

## Define a $k$-space trajectory, a list of $\omega$, Gaussian broadening factor $\sigma$, and temperature $T$ for computing $\mathcal{S}^{ab}({\bm k}, \omega)$

In [ ]:
kc = sptk.k_cut(lattice = latt, density = 200)
kc.add_k(k_new = [-1.0 / 3.0, -1.0 / 3.0])  # -K
kc.add_k(k_new = [0.0, 0.0])                # Gamma
xtic_G = kc.len_list[-1]                    # x-axis position for Gamma-point (for plotting)
kc.add_k(k_new = [1.0 / 3.0, 1.0 / 3.0])    # K

ω_min = -6.0
ω_max =  6.0
num_ω =  1200
ω_list = np.arange(ω_min, ω_max, (ω_max - ω_min) / num_ω, dtype = np.float64).tolist()

# Gaussian broadening factor
FWHM  = 0.05
sigma = FWHM / 2.35482

T = 0.5 * J


## Compute finite-$T$ 1-magnon DSSF $\mathcal{S}_{1}^{ab}({\bm k},\omega)$

In [ ]:

omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

start = time.perf_counter()
total_k = kc.size()
for index_k in range(total_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        disp0, Sxx0, Syy0, Szz0, \
        SxyPyx_R0, SyzPzy_R0, SzxPxz_R0, \
        SxyMyx_I0, SyzMzy_I0, SzxMxz_I0 \
        = sptk.DSSF_LSW(model = hamiltonian, R = R, T = T,
                        k = k, omega_list = ω_list,
                        eval_1magnon = True, maxeval_2magnon = 0,
                        maxeval_3magnon = 0, maxeval_0magnon = 0,
                        broadening = "Gaussian", sigma_or_eta = sigma,
                        epsilon = 1.0e-5)
        omega_k.append(disp0)
        Sxx.append(Sxx0)
        Syy.append(Syy0)
        Szz.append(Szz0)
        SxyPyx_R.append(SxyPyx_R0)
        SyzPzy_R.append(SyzPzy_R0)
        SzxPxz_R.append(SzxPxz_R0)
        SxyMyx_I.append(SxyMyx_I0)
        SyzMzy_I.append(SyzMzy_I0)
        SzxMxz_I.append(SzxMxz_I0)
    except Exception as e:
        print(f"error: {e}, k = {k}")
print()
end = time.perf_counter()
print(f"Used {end - start:.6f}s")

In [ ]:
# save and read data
_, num_disp = np.array(omega_k).shape
x = kc.len_list

INSdatafile = f"INS_1magnon_T{T}K_triangular.txt"
Dispdatafile = f"Disp_1magnon_T{T}K_triangular.txt"

axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

# save data
np.savetxt(INSdatafile,
         np.column_stack((axis_X.flatten(), axis_Y.flatten(), Intensity.flatten())),
           header="#(1)\t(2)\t(3)\n#k\tomega\tIntensity")


np.savetxt(Dispdatafile, np.column_stack((x, np.array(omega_k))),
           header="\t".join([f"({i})" for i in range(num_disp + 1)] + ["\nk"] + [f"omega_{i+1}" for i in range(num_disp)]),
           comments='#')

# read data
# num_rows = len(ω_list)
# num_cols = len(x)
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))

# data = np.loadtxt(Dispdatafile)
# omega_k = data[:, 1:]


## Plot 1-magnon DSSF $\mathcal{S}_{1}({\bm k}, \omega) \equiv \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_1^{yy}({\bm k}, \omega) + \mathcal{S}_1^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.1
vmax = 100
cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
# imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", vmin = 0, vmax = 100)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", norm=colors.LogNorm(vmin=vmin, vmax=vmax))

xmin = x[0]
xmax = x[-1]
ymax = ω_max
ymin = ω_min

ax.set_xlim(x[0], x[-1])
ax.set_ylim(ymin, ymax)
ax.set_xticks([x[0], xtic_G, x[-1]], [r"$(-\frac{1}{3},-\frac{1}{3})$", r"$(0,0)$", r"$(\frac{1}{3},\frac{1}{3})$"])
ax.set_yticks([-6, -3, 0, 3, 6])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.63 * xmax, -0.7 * ymax, r'$\mathcal{S}_{1}(\bm{k},\omega)$', fontsize = 15)
ax.text(0.48 * xmax, -0.9 * ymax, fr'${vmin}$', fontsize = 15)
ax.text(0.90 * xmax, -0.9 * ymax, fr'${vmax}$', fontsize = 15)

cax = ax.inset_axes([0.57 * xmax, -0.9 * ymax, 0.32 * (xmax-xmin), 0.05 * (ymax - ymin)], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0.1, 1, 10, 100])
cbar.set_ticklabels(['', '', '', ''])
cbar.ax.tick_params(which = 'both', direction = 'in')
# cbar.ax.minorticks_off()

plt.savefig(f"Fig_1Magnon_T{T}K_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute finite-$T$ 2-magnon DSSF $\mathcal{S}_{2}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

def process_S2(index_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DSSF_LSW(model = hamiltonian, R = R, T = T,
                             k = k, omega_list = ω_list,
                             eval_1magnon = False, maxeval_2magnon = 100000,
                             maxeval_3magnon = 0, maxeval_0magnon = 0,
                             broadening = "Gaussian", sigma_or_eta = sigma,
                             epsilon = 1.0e-5)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_S2 = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_S2)(index_k) for index_k in range(total_k))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_S2)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

In [ ]:
# save and read data
_, num_disp = np.array(omega_k).shape
x = kc.len_list

INSdatafile = f"INS_2magnon_T{T}K_triangular.txt"
Dispdatafile = f"Disp_2magnon_T{T}K_triangular.txt"

axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

# save data
np.savetxt(INSdatafile,
         np.column_stack((axis_X.flatten(), axis_Y.flatten(), Intensity.flatten())),
           header="#(1)\t(2)\t(3)\n#k\tomega\tIntensity")


np.savetxt(Dispdatafile, np.column_stack((x, np.array(omega_k))),
           header="\t".join([f"({i})" for i in range(num_disp + 1)] + ["\nk"] + [f"omega_{i+1}" for i in range(num_disp)]),
           comments='#')

# read data
# num_rows = len(ω_list)
# num_cols = len(x)
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))

# data = np.loadtxt(Dispdatafile)
# omega_k = data[:, 1:]


## Plot 2-magnon DSSF $\mathcal{S}_{2}({\bm k}, \omega) \equiv \mathcal{S}_{2}^{xx}({\bm k}, \omega) + \mathcal{S}_{2}^{yy}({\bm k}, \omega) + \mathcal{S}_{2}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.01
vmax = 10

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", norm=colors.LogNorm(vmin=vmin, vmax=vmax))
# uncomment the following line if you like to overplot the 1-magnon dispersion
# ax.plot(x, np.array(omega_k), 'r--')

xmin = x[0]
xmax = x[-1]
ymax = ω_max
ymin = ω_min

ax.set_xlim(x[0], x[-1])
ax.set_ylim(ymin, ymax)
ax.set_xticks([x[0], xtic_G, x[-1]], [r"$(-\frac{1}{3},-\frac{1}{3})$", r"$(0,0)$", r"$(\frac{1}{3},\frac{1}{3})$"])
ax.set_yticks([-6, -3, 0, 3, 6])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.63 * xmax, -0.7 * ymax, r'$\mathcal{S}_{2}(\bm{k},\omega)$', fontsize = 15)
ax.text(0.455 * xmax, -0.9 * ymax, r'$10^{-2}$', fontsize = 15)
ax.text(0.92 * xmax, -0.9 * ymax, fr'${vmax}$', fontsize = 15)

cax = ax.inset_axes([0.58 * xmax, -0.9 * ymax, 0.32 * (xmax-xmin), 0.05 * (ymax - ymin)], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0.01, 0.1, 1, 10])
cbar.set_ticklabels(['', '', '', ''])
cbar.ax.tick_params(which = 'both', direction = 'in')

plt.savefig(f"Fig_2Magnon_T{T}K_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute finite-$T$ 3-magnon DSSF $\mathcal{S}_{3}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

def process_S3(index_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DSSF_LSW(model = hamiltonian, R = R, T = T,
                             k = k, omega_list = ω_list,
                             eval_1magnon = False, maxeval_2magnon = 0,
                             maxeval_3magnon = 1000000, maxeval_0magnon = 0,
                             broadening = "Gaussian", sigma_or_eta = sigma,
                             epsilon = 1.0e-5)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_S3 = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_S3)(index_k) for index_k in range(total_k))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_S3)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

In [ ]:
# save and read data
_, num_disp = np.array(omega_k).shape
x = kc.len_list

INSdatafile = f"INS_3magnon_T{T}K_triangular.txt"
Dispdatafile = f"Disp_3magnon_T{T}K_triangular.txt"

axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

# save data
np.savetxt(INSdatafile,
         np.column_stack((axis_X.flatten(), axis_Y.flatten(), Intensity.flatten())),
           header="#(1)\t(2)\t(3)\n#k\tomega\tIntensity")


np.savetxt(Dispdatafile, np.column_stack((x, np.array(omega_k))),
           header="\t".join([f"({i})" for i in range(num_disp + 1)] + ["\nk"] + [f"omega_{i+1}" for i in range(num_disp)]),
           comments='#')

# read data
# num_rows = len(ω_list)
# num_cols = len(x)
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))

# data = np.loadtxt(Dispdatafile)
# omega_k = data[:, 1:]


## Plot 3-magnon DSSF $\mathcal{S}_{3}({\bm k}, \omega) \equiv \mathcal{S}_{3}^{xx}({\bm k}, \omega) + \mathcal{S}_3^{yy}({\bm k}, \omega) + \mathcal{S}_3^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.01
vmax = 10

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", norm=colors.LogNorm(vmin=vmin, vmax=vmax))
# uncomment the following line if you like to overplot the 1-magnon dispersion
# ax.plot(x, np.array(omega_k), 'r--')

xmin = x[0]
xmax = x[-1]

ymin = ω_min
ymax = ω_max

ax.set_xlim(x[0], x[-1])
ax.set_ylim(ymin, ymax)
ax.set_xticks([x[0], xtic_G, x[-1]], [r"$(-\frac{1}{3},-\frac{1}{3})$", r"$(0,0)$", r"$(\frac{1}{3},\frac{1}{3})$"])
ax.set_yticks([-6, -3, 0, 3, 6])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.63 * xmax, -0.7 * ymax, r'$\mathcal{S}_{3}(\bm{k},\omega)$', fontsize = 15)
ax.text(0.455 * xmax, -0.9 * ymax, r'$10^{-2}$', fontsize = 15)
ax.text(0.92 * xmax, -0.9 * ymax, fr'${vmax}$', fontsize = 15)

cax = ax.inset_axes([0.58 * xmax, -0.9 * ymax, 0.32 * (xmax-xmin), 0.05 * (ymax - ymin)], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0.01, 0.1, 1, 10])
cbar.set_ticklabels(['', '', '', ''])
cbar.ax.tick_params(which = 'both', direction = 'in')

plt.savefig(f"Fig_3Magnon_T{T}K_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()